In [5]:
import os
import time
import pickle
import itertools
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt

from genEM3.data.wkwdata import WkwData,DataSource
from genEM3.util.path import get_data_dir
import genEM3.data.annotation as annotation 
from genEM3.data.annotation import Widget

In [7]:
# json files to reannotate
json_dirs = {'train':os.path.join(get_data_dir(), 'dense_3X_10_10_2_um/original_merged_with_myelin_v01.json'),
             'test':os.path.join(get_data_dir(), '10x_test_bboxes/10X_9_9_1_um_with_myelin_v01.json')}

In [11]:
# Loaded the json filejoin for the dataset
json_dir = json_dirs.get('train')
config = WkwData.config_wkwdata(json_dir)
dataset = WkwData.init_from_config(config)

# Get a set of data sources with the normal bounding boxes to create a patch wise detaset and a larger bounding box for annotation
margin = 35
roi_size = 140
source_dict = annotation.patch_source_list_from_dataset(dataset=dataset,
                                                        margin=margin,
                                                        roi_size=roi_size)
# Creat the datasets
dataset_dict = dict.fromkeys(source_dict)
for key in source_dict:
    cur_source = source_dict[key]
    cur_patch_shape = tuple(cur_source[0].input_bbox[3:6])
    cache_RAM = True if key == 'large' else False
    cur_config = WkwData.config_wkwdata(datasources_json_path=None,
                                        input_shape=cur_patch_shape,
                                        output_shape=cur_patch_shape,
                                        cache_RAM=cache_RAM)
    dataset_dict[key] = WkwData.init_from_config(cur_config, source_dict[key])


t
Filling caches ... data source 12636/12837 input
Filling caches ... data source 12636/12837 target
Filling caches ... data source 12637/12837 input
Filling caches ... data source 12637/12837 target
Filling caches ... data source 12638/12837 input
Filling caches ... data source 12638/12837 target
Filling caches ... data source 12639/12837 input
Filling caches ... data source 12639/12837 target
Filling caches ... data source 12640/12837 input
Filling caches ... data source 12640/12837 target
Filling caches ... data source 12641/12837 input
Filling caches ... data source 12641/12837 target
Filling caches ... data source 12642/12837 input
Filling caches ... data source 12642/12837 target
Filling caches ... data source 12643/12837 input
Filling caches ... data source 12643/12837 target
Filling caches ... data source 12644/12837 input
Filling caches ... data source 12644/12837 target
Filling caches ... data source 12645/12837 input
Filling caches ... data source 12645/12837 target
Filling 

In [9]:
# Annotation
button_names = [['No', 'Yes'], ['No', 'Yes']]
target_classes = ['Myelin', 'Debris']
w = annotation.Widget(dataset=dataset_dict['large'], 
                      index_range=range(len(dataset_dict['large'])),
                      button_names = button_names,
                      target_classes = target_classes,
                      margin=margin, 
                      roi_size = roi_size)
# Update annotation from the multiclass labels in the dataset
w.update_annotation_from_multiclass()
w.show_widget()